# Prepare ESM2 pretrain

We choose the esm2_t33_650M_UR50D model from this [hub](https://github.com/facebookresearch/esm#available-models).

For fit esm2 model input, we set the amino acid seqs MAX_LEN = 1022 (+ cls, eos == 1024).

For each protein(length = m), it will generate a feature mat, shape=(m, 1280). 

By mean this mat, it output a feature vec, dim=(1280).

Cuz the limitation of the memory, we only compute one protein each time.

In [1]:
import torch
import esm
import pandas as pd

In [2]:
torch.set_num_threads(2)

In [3]:
# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results 

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bia

In [16]:
from tqdm import tqdm
import pickle

def get_esm_pretrain(model, df_dir, db_name, sep=' ', header=None, col_names=['drug_id', 'prot_id', 'drug_smile', 'prot_seq', 'label']):
    df = pd.read_csv(df_dir, sep=sep)
    df.columns = col_names
    df.drop_duplicates(subset='prot_id', inplace=True)
    prot_ids = df['prot_id'].tolist()
    prot_seqs = df['prot_seq'].tolist()
    data = []
    prot_size = len(prot_ids)
    for i in range(prot_size):
        seq_len = min(len(prot_seqs[i]),1022)
        data.append((prot_ids[i], prot_seqs[i][:seq_len]))
    
    emb_dict = {}
    emb_mat_dict = {}
    length_target = {}

    for d in tqdm(data):
        prot_id = d[0]
        batch_labels, batch_strs, batch_tokens = batch_converter([d])
        batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
        # Extract per-residue representations (on CPU)
        with torch.no_grad():
            results = model(batch_tokens, repr_layers=[33], return_contacts=True)
        token_representations = results["representations"][33].numpy()

        sequence_representations = []
        for i, tokens_len in enumerate(batch_lens):
            sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

        emb_dict[prot_id] = sequence_representations[0]
        emb_mat_dict[prot_id] = token_representations[0]
        length_target[prot_id] = len(d[1])
    
    dump_data = {
        "dataset": db_name,
        "vec_dict": emb_dict,
        "mat_dict": emb_mat_dict,
        "length_dict": length_target
    }    
    with open(f'./{db_name}_esm_pretrain.pkl', 'wb+') as f:
        pickle.dump(dump_data, f)    

In [19]:
db_name = 'case_study'
df_dir = '/homeb/tangwuguo/projects/ProVecDTA/data/case_prots.csv'
col_names = ['prot_id', 'prot_seq']
get_esm_pretrain(model, df_dir, db_name, sep='\t', header=0, col_names=col_names)

100%|██████████| 6/6 [02:35<00:00, 25.96s/it]


In [23]:
df = pd.read_csv(df_dir, sep='\t')
df.columns = col_names
df

,prot_id,prot_seq
0,3C-like proteinase (accession YP_009725301.1),SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTS...
1,RNA-dependent RNA polymerase (accession YP_009...,SADAQSFLNRVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFA...
2,helicase (accession YP_009725308.1),AVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVN...
3,3'-to-5' exonuclease (accession YP_009725309.1),AENVTGLFKDCSKVITGLHPTQAPTHLSVDTKFKTEGLCVDIPGIP...
4,endoRNAse (accession YP_009725310.1),SLENVAFNVVNKGHFDGQQGEVPVSIINNTVYTKVDGVDVELFENK...
5,2'-O-ribose methyltransferase (accession YP_00...,SSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMNVAK...


In [6]:
db_names = ['davis', 'kiba', 'metz']
df_dirs = [r'/home/tangwuguo/datasets/davis.txt', r'/home/tangwuguo/datasets/kiba.txt', r'/home/tangwuguo/datasets/metz.txt']

for i in range(1,2):
    print(f'Compute {df_dirs[i]} protein pretrain feature by esm2.')
    get_esm_pretrain(model, df_dirs[i], db_names[i])

Compute /home/tangwuguo/datasets/kiba.txt protein pretrain feature by esm2.


100%|██████████| 229/229 [42:19<00:00, 11.09s/it]


# Mol2Vec pretrain
Input drug smiles seq, firstly it will compute the sub-structure of this drug.

For one SMILES, sub-strutures num=m, it outputs a (m,300) feature mat and a 300-dim feature vector.

In [20]:
import numpy as np 
import pandas as pd 
import pickle
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence
from rdkit import Chem
from tqdm import tqdm

In [35]:
def get_mol2vec(mol2vec_dir, df_dir, db_name, sep=' ', header=None, col_names=['drug_id', 'prot_id', 'drug_smile', 'prot_seq', 'label'], embedding_dimension=300, is_debug=False):
    mol2vec_model = word2vec.Word2Vec.load(mol2vec_dir)
    
    df = pd.read_csv(df_dir, header=header, sep=sep)
    df.columns = col_names
    df.drop_duplicates(subset='drug_id', inplace=True)    
    drug_ids = df['drug_id'].tolist()
    drug_seqs = df['drug_smile'].tolist()
    
    emb_dict = {}
    emb_mat_dict = {}
    length_dict = {}
    
    percent_unknown = []
    bad_mol = 0
    
    # get pretrain feature
    for idx in tqdm(range(len(drug_ids))):
        flag = 0
        mol_miss_words = 0
        
        drug_id = str(drug_ids[idx])
        molecule = Chem.MolFromSmiles(drug_seqs[idx])
        length_dict
        try:
            # Get fingerprint from molecule
            sub_structures = mol2alt_sentence(molecule, 2)
        except Exception as e: 
            if is_debug: 
                print (e)
            percent_unknown.append(100)
            continue    
                
        emb_mat = np.zeros((len(sub_structures), embedding_dimension))
        length_dict[drug_id] = len(sub_structures)
                
        for i, sub in enumerate(sub_structures):
            # Check to see if substructure exists
            try:
                emb_dict[drug_id] = emb_dict.get(drug_id, np.zeros(embedding_dimension)) + mol2vec_model.wv[sub]  
                emb_mat[i] = mol2vec_model.wv[sub]  
            # If not, replace with UNK (unknown)
            except Exception as e:
                if is_debug : 
                    print ("Sub structure not found")
                    print (e)
                emb_dict[drug_id] = emb_dict.get(drug_id, np.zeros(embedding_dimension)) + mol2vec_model.wv['UNK']
                emb_mat[i] = mol2vec_model.wv['UNK']                
                flag = 1
                mol_miss_words = mol_miss_words + 1        
        emb_mat_dict[drug_id] = emb_mat
        
        percent_unknown.append((mol_miss_words / len(sub_structures)) * 100)
        if flag == 1:
            bad_mol = bad_mol + 1
            
    print(f'All Bad Mol: {bad_mol}, Avg Miss Rate: {sum(percent_unknown)/len(percent_unknown)}%')
        
    dump_data = {
        "dataset": db_name,
        "vec_dict": emb_dict,
        "mat_dict": emb_mat_dict,
        "length_dict": length_dict
    }    
    with open(f'./{db_name}_mol_pretrain.pkl', 'wb+') as f:
        pickle.dump(dump_data, f)    

In [36]:
mol2vec_dir = '/home/tangwuguo/datasets/model_300dim.pkl'
db_names = 'case_study'
df_dir = '/homeb/tangwuguo/projects/ProVecDTA/data/case_drugs.csv'

get_mol2vec(mol2vec_dir, df_dir, db_name, sep=',', header=0, col_names=['drug_id', 'drug_smile'])

100%|██████████| 3137/3137 [00:13<00:00, 229.82it/s]


All Bad Mol: 3134, Avg Miss Rate: 27.784826124584857%


In [ ]:
mol2vec_dir = '/home/tangwuguo/datasets/model_300dim.pkl'
db_names = ['davis', 'kiba', 'metz']
df_dirs = [r'/home/tangwuguo/datasets/davis.txt', r'/home/tangwuguo/datasets/kiba.txt', r'/home/tangwuguo/datasets/metz.txt']

for i in range(3):
    print(f'Compute {db_names[i]} drug pretrain feature by protvec.')
    get_esm_pretrain(mol2vec_dir, df_dirs[i], db_names[i])

# ProtVec pretrain
Inuput protein's amino acid seq, len = m

Output feature mat(m,100), feature vector(100)

In [ ]:
def get_protvec(protvec_dir, df_dir, db_name, col_names=['drug_id', 'prot_id', 'drug_smile', 'prot_seq', 'label'], embedding_dimension=100, is_debug=False):        
    protvec_model = pd.read_csv(protvec_dir, delimiter = '\t')
    trigram_dict = {}
    for idx, row in tqdm(protvec_model.iterrows()):
        trigram_dict[row['words']] = protvec_model.iloc[idx, 1:].values.astype(np.float)
    trigram_list = set(trigram_dict.keys())
    
    df = pd.read_csv(df_dir, header=None, sep=' ')
    df.columns = col_names
    df.drop_duplicates(subset='prot_id', inplace=True)    
    prot_ids = df['prot_id'].tolist()
    prot_seqs = df['prot_seq'].tolist()
    
    emb_dict = {}
    emb_mat_dict = {}
    length_3mer_target = {}


    # get pretrain feature
    for idx in tqdm(range(len(prot_ids))):
        n = 3
        target = prot_seqs[idx]
        prot_id = str(prot_ids[idx])
        split_by_three = [target[i : i + n] for i in range(0, len(target), n)]
        mer_len = len(split_by_three)
        length_3mer_target[prot_id] = mer_len
        
        emb_mat = np.zeros((mer_len, embedding_dimension))
        for i, trigram in enumerate(split_by_three): 
            if len(trigram) == 2: 
                trigram = "X" + trigram
            elif len(trigram) == 1:
                trigram = "XX" + trigram
            if trigram in trigram_list:
                emb_dict[prot_id] = emb_dict.get(prot_id, np.zeros(embedding_dimension))+ trigram_dict[trigram]
                emb_mat[i] = trigram_dict[trigram]
        emb_mat_dict[prot_id] = emb_mat
    # 存储pretrain
    dump_data = {
        "dataset": db_name,
        "vec_dict": emb_dict,
        "mat_dict": emb_mat_dict,
        "length_dict": length_3mer_target
    }    
    with open(f'./{db_name}_prot_pretrain.pkl', 'wb+') as f:
        pickle.dump(dump_data, f)    
    

In [ ]:
mol2vec_dir = '/home/tangwuguo/datasets/protVec_100d_3grams.csv'
db_names = ['davis', 'kiba', 'metz']
df_dirs = [r'/home/tangwuguo/datasets/davis.txt', r'/home/tangwuguo/datasets/kiba.txt', r'/home/tangwuguo/datasets/metz.txt']

for i in range(3):
    print(f'Compute {db_names[i]} protein pretrain feature by protvec.')
    get_esm_pretrain(mol2vec_dir, df_dirs[i], db_names[i])